In [ ]:
from peft import PromptTuningConfig, PromptTuningInit, TaskType, get_peft_model
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    GPTNeoXForCausalLM,
    Trainer,
    TrainingArguments
)
from torch.utils.data import DataLoader
import datasets

In [ ]:
dataset = datasets.load_from_disk("/fsx/proj-chemnlp/data/EleutherAI/pythia-160m/marianna13/chemrxiv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="EleutherAI/pythia-160m",
    revision="main",
)
tokenizer.add_special_tokens({"pad_token": "<|padding|>"})

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
dl = DataLoader(dataset, batch_size=2, collate_fn=data_collator)

In [ ]:
model = GPTNeoXForCausalLM.from_pretrained(
    pretrained_model_name_or_path="EleutherAI/pythia-160m",
    revision="main",
)

In [ ]:
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=10,
    prompt_tuning_init_text=" ",
    tokenizer_name_or_path="EleutherAI/pythia-160m",
)
peft_model = get_peft_model(model, peft_config)

In [ ]:
for i, (p1,p2) in enumerate(zip(model.parameters(), peft_model.parameters())):
    import torch
    if not torch.equal(p1,p2):
        print(i, False)
    else:
        print(i)

In [ ]:
for b in dl:
    token_types = b.pop('token_type_ids')
    break

In [ ]:
out = model(**b)
out['loss']

In [ ]:
out = peft_model(**b)
out['loss']

In [ ]:
all([p.requires_grad for p in model.parameters()]), all([p.requires_grad for p in peft_model.parameters()])

In [ ]:
split_dataset = dataset.train_test_split(test_size=0.025)

In [ ]:
training_args = TrainingArguments(
        output_dir='./',
        num_train_epochs=1,
        learning_rate=3e-4,
        evaluation_strategy='steps',
        logging_steps=1,
        eval_steps=50,
        dataloader_num_workers=4,
        bf16=True,
        fp16=False,
        per_device_train_batch_size=28,
        per_device_eval_batch_size=28,
        report_to="none",
    )

In [ ]:
trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=split_dataset["train"],
        eval_dataset=split_dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

In [ ]:
trainer.train()